In [30]:
#Заданные величины
U_m = 4.4
R_n = 0.55 * 10**3
R_g = 1.3 * 10**3
dT = 60
h_fe = (40, 120)
#Номинальный ряд
E24 = (10, 11, 12, 13, 15, 16, 18, 20, 22, 24, 27, 30, 33, 36, 39, 43, 47, 51, 56, 62, 68, 75, 82, 91)

# Пункт 4

In [31]:
#Рабочая точка
dU_n = 1
dU_t = 1
U_ke = 7
I_k = 24 * 10**-3
E_p = 20
print(U_ke, I_k* 10**3, E_p) 

7 24.0 20


# Пункт 5

In [32]:
R_dc = (E_p-abs(U_ke))/I_k
for n in E24:
    if R_dc < n*10:
        R_e = n*10
        break
R_ac = R_e*R_n/(R_e+R_n)
print(R_dc, R_e)

541.6666666666666 560


# Пункт 6


In [33]:
dI_k = dU_t/R_e
xi = 1.8*10**-3
beta = (h_fe[1]+h_fe[0])/2
dbeta = h_fe[1]-beta
I_b = I_k/beta
R_b = (beta*(dI_k*R_e-xi*dT))/(I_b*dbeta-dI_k) - R_e
print(dI_k * 10**3)

I_e = I_b + I_k

R1 = E_p/(I_e*R_e) * R_b
R2 = R_b*R1/(R1-R_b)
print(R1, R2)
for n in E24:
    if R1/1000 not in E24 and R1 < n*10**3:
        R1 = n*10**3
    if R2/1000 not in E24 and R2 < n*10**3:
        R2 = n*10**3
print(R1, R2)

1.7857142857142856
9444.876111542777 20107.301959617358
10000 22000


# Пункт 7

In [34]:
U_be = -0.7
E_b = E_p * R2/(R1+R2)
I_b = (E_b-U_be)/(R_b+(1+beta)*R_e)
I_k = beta*I_b
I_e = I_k + I_b
U_ke = E_p - I_e*R_e
print(U_be, I_b * 10**6, U_ke, I_k * 10**3)

-0.7 279.03136072940976 7.343137477313974 22.32250885835278
